# Genetic Algorithm Exercise

## Description: General Idea

The goals of this script is to produce "offspring" whose color visually approaches the one given by the user as input. It attempts to simulate the evolution of organisms, following basic genetic rules.

#### Input
The user chooses the initial number of individuals and the number of generations to be computed, as well as the target color of the individuals.

#### Output
The script generates in a ".dot" file the description of a directed graph, in which the nodes have a color and shape that correspond to their "genes".

#### Generation
The initial pool is generated with a random RGB color and a random shape - 7/15 chance to be a circle (both dominant genes - RR), 5/15 egg (one dominant and one recessive - Rr), 3/15 star (both recessive genes - rr).

#### Selection
The individuals are sorted based on the distance between their color and the color chosen by the user. Then, individuals from the first 1/5 of the sorted pool are paired to generate offspring.

#### Crossover
The shape follows the rules of dominant-recessive genes, whereas the color follows the rule of codominance. I compute the color as the average of the colors of the parents, on R, G, and B separately. At crossover, one of the parents has a chance of 1/4 to die and the other has a chance of 1/3.



In [11]:
import random
import math
from math import floor, sqrt

random.seed()

#initialize file
outf = open ("evolution.dot", "w")
outf.write("digraph {\n")

#get user preferences
poolSize = input ("Enter the initial number of individuals: ")
steps = input("Enter the number of generations you want to create: ")
goalR = input("Next, enter your favorite color.\nR: ")
goalG = input("G: ")
goalB = input("B: ")

class individual:
    index = 0
    
    def __init__(self): 
        self.red = random.randint(0, 255)
        self.green = random.randint(0, 255)
        self.blue = random.randint(0, 255)
        self.shape = random.choice([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2])
    

def rgb2hex(r,g,b):
    hexCode = "#{:02x}{:02x}{:02x}".format(r,g,b)
    return hexCode

def phenotype(indiv):
    cellDesign = '\t' + str(indiv.index) + ' [shape='
    
    if (indiv.shape == 0):
        cellDesign = cellDesign + 'circle' #DD
    elif (indiv.shape == 1):
        cellDesign = cellDesign + 'egg' #DR
    else:
        cellDesign = cellDesign + 'star' #RR
    
    cellDesign = cellDesign + ',color="' + rgb2hex(indiv.red, indiv.green, indiv.blue) + '",style=filled,label='
    cellDesign = cellDesign + str(indiv.index) + '];\n'
    
    return cellDesign
    
#generate initial pool
pool = [] #empty list - you have to append things to it.
for i in range(int(poolSize)):
    x = individual()
    pool.append(x)
    pool[i].index = i
    outf.write(phenotype(pool[i]))
    outf.write("\t" + str(i) + ";\n")
crtIndex = int(poolSize)

def crossover(parent1, parent2, sonIndex): 
    son = individual ()
    son.index = sonIndex
    
    #inheritance rule for the codominant gene (color, separated in 3 genes corresponding to R, G and B)
    son.red = int( (parent1.red + parent2.red) / 2 )
    son.green = int( (parent1.green + parent2.green) / 2 )
    son.blue = int( (parent1.blue + parent2.blue) / 2 )
    
    if (parent1.shape > parent2.shape):
        parent1, parent2 = parent2, parent1
    
    #inheritance rules (D = dominant gene, R = recessive gene; 0 is DD, 1 is DR, 2 is RR)
    if (parent1.shape == 0 and parent2.shape == 0):
        son.shape = 0
    elif (parent1.shape == 0 and parent2.shape == 2):
        son.shape = 1
    elif (parent1.shape == 0 and parent2.shape == 1):
        son.shape = random.choice([0, 1])
    elif (parent1.shape == 1 and parent2.shape == 1):
        son.shape = random.choice([0, 1, 1, 2])
    elif (parent1.shape == 2 and parent2.shape == 2):
        son.shape = 2
    
    #add son in file
    outf.write ('\t' + str(parent1.index) + "->" + str(son.index) + ";\n")
    outf.write ('\t' + str(parent2.index) + "->" + str(son.index) + ";\n")
    outf.write (phenotype (son))
    
    #remove parents from the pool
    #the first one has a 25% chance to be removed
    if (random.choice([0, 1, 1, 1]) == 0):
        pool.remove(parent1)
    
    #the second one has a 33% chance to be removed
    if (random.choice([0, 1, 1]) == 0):
        pool.remove(parent2)

    #add son to the pool
    pool.append(son)
    
def colorDistance(r1, g1, b1, r2 = int(goalR), g2 = int(goalG), b2 = int(goalB)):
    dist = math.floor( math.sqrt( 2 * ( (r1 - r2) ** 2) + 4 * ( (g1 - g2) ** 2) + 3 * ( (b1 - b2) ** 2) ) )
    return dist

def colorCriterion(a):
    dista = colorDistance(a.red, a.green, a.blue)
    return dista

for i in range(int(steps)):
    pool.sort(key=colorCriterion)
    
    for j in range(math.floor(len(pool) / 5)):
        crossover(pool[j], pool[j + 1], crtIndex)
        crtIndex+=1
    
#close the file
outf.write("}\n")
outf.close()

Enter the initial number of individuals: 30
Enter the number of generations you want to create: 8
Next, enter your favorite color.
R: 73
G: 106
B: 231


### Example Outputs

<img src="evolutionAndreea0.jpg"/>

<img src="evolutionAndreea1.jpg"/>

<img src="evolutionAndreea2.jpg"/>

<img src="evolutionAndreea3.jpg"/>

##### References

https://stackoverflow.com/questions/3380726/converting-a-rgb-color-tuple-to-a-six-digit-code-in-python

https://www.compuphase.com/cmetric.htm
